# nature DQN

value based;

In [1]:
import argparse
import os
import time

import gym
import tensorflow as tf
import tensorlayer as tl
import pandas as pd
import numpy as np

# %run 

UsageError: Line magic function `%tf.__version__` not found.


## 前置知识

### DP, MC, TD
强化学习中策略可以分为:
- 目标策略(target policy): 智能体要学习的策略.
- 行为策略(behavior policy): 智能体与环境交互的策略,即用于生成行为的策略.

广义策略迭代:
- 策略评估(policy evaluation): estimate the value function $v_{\pi}$ for a given policy $\pi$.
- 策略改进(policy improvement): find an optimal policy.

on-policy指目标和行为策略相同, off-policy指目标和行为策略不同.  

TD方法介于Monte Carlo(MC)方法和Dynamic Programming(DP)方法之间, 既有MC方法直接从原始经验中学习的优点, 也有DP方法自举(bootstrap, 即用一个状态价值估计另一个状态价值)的优点.

DP更新:
$$V\left(S_{t}\right) \leftarrow E_{\pi}\left[R_{t+1}+\gamma V\left(S_{t+1}\right)\right]$$

MC更新:
$$V\left(S_{t}\right) \leftarrow V\left(S_{t}\right)+\alpha\left[G_{t}-V\left(S_{t}\right)\right]$$

TD更新: 
$$V\left(S_{t}\right) \leftarrow V\left(S_{t}\right)+\alpha\left[R_{t+1}+\gamma V\left(S_{t+1}\right)-V\left(S_{t}\right)\right]$$


### SARSA, Q-learning
Q-learning是一种off-policy TD方法.Q-learning 用 Q-Table 来存储每个状态动作对的 Q 值，若状态和动作空间是离散且维度不高时，是比较有效的；若状态和动作空间是高维连续时，就会出现“curse of dimensionality”，即随着维数的增加，计算量呈指数倍增长。

既然我们无法用一个表格来精确地存储与表示 $Q$ 值，我们可以用一个参数化 (parameterized) 函数来近似地表示动作值函数 $Q(s,a)$. 

SARSA update:
$$Q\left(S_{t}, A_{t}\right) \leftarrow Q\left(S_{t}, A_{t}\right)+\alpha\left[R_{t+1}+\gamma Q\left(S_{t+1}, A_{t+1}\right)-Q\left(S_{t}, A_{t}\right)\right]$$

Q-learning update:
$$Q\left(S_{t}, A_{t}\right) \leftarrow Q\left(S_{t}, A_{t}\right)+\alpha\left[R_{t+1}+\gamma \max _{a} Q\left(S_{t+1}, a\right)-Q\left(S_{t}, A_{t}\right)\right]$$


## 1. 经验回放(Experience Replay)

DQN 面临着几个挑战：

- 深度学习需要大量带标签的训练数据；
- 强化学习从 scalar reward 进行学习，但是 reward 经常是 sparse, noisy, delayed；
- 深度学习假设样本数据是独立同分布的，但是强化学习中采样的数据是强相关的

因此，DQN 采用经验回放（Experience Replay）机制，将训练过的数据进行储存到 Replay Buffer 中，以便后续从中随机采样进行训练，好处就是：
1. 数据利用率高；
2. 减少连续样本的相关性，从而减小方差（variance）。

## 2. 目标网络

![DQN流程图](./images/ZKM%60P(HWELOB64KMFN1%5D379.png)

# Reference